# Generate more data

In [40]:
import numpy as np
import time

## Read input data and generate more data
Generate more data:
- Vertical reflection
- Single pixel translations (up, down, left, right) with wrapping

Will results in (1 original) + (1 reflection) + (4 orig trans) + (4 refl trans) = 10x the data.

In [41]:
data_bacon_file = open("../processor/data/matrix/bacon.dat", "rb")
bacon_data = np.fromfile(data_bacon_file, dtype=np.uint8)
# 100 images, 4 channels (R,G,B,L), 64x64 pixels
bacon_data = bacon_data.reshape((100, 1, 128, 128))
data_bacon_file.close()

In [42]:
data_kevin_file = open("../processor/data/matrix/kevin.dat", "rb")
kevin_data = np.fromfile(data_kevin_file, dtype=np.uint8)
# 100 images, 4 channels (R,G,B,L), 64x64 pixels
kevin_data = kevin_data.reshape((100, 1, 128, 128))
data_kevin_file.close()

In [43]:
data_not_file = open("../processor/data/matrix/not.dat", "rb")
not_data = np.fromfile(data_not_file, dtype=np.uint8)
# 200 images, 4 channels (R,G,B,L), 64x64 pixels
not_data = not_data.reshape((200, 1, 128, 128))
data_not_file.close()

In [44]:
# Reflection about vertical axis
def reflect_about_vertical(data):
    reflected_images = []
    for image in data:
        reflected_channels = []
        for channel in image:
            reflected_rows = []
            for row in channel:
                reflected_rows.append(row[::-1].tolist())
            reflected_channels.append(reflected_rows)
        reflected_images.append(reflected_channels)
    return np.asarray(reflected_images, dtype=np.uint8)

In [45]:
refl_bacon_data = reflect_about_vertical(bacon_data)
refl_kevin_data = reflect_about_vertical(kevin_data)
refl_not_data = reflect_about_vertical(not_data)

In [46]:
# Translation with wrapping
def translate(data, direction=None):
    if direction is None or direction not in ["up", "down", "left", "right"]:
        return None
    
    trans_images = []
    for image in data:
        trans_channels = []
        for channel in image:
            
            if direction == "right":
                trans_rows = []
                for row in channel:
                    trans_rows.append([row[-1]] + row[:-1].tolist())
                trans_channels.append(trans_rows)
            elif direction == "left":
                trans_rows = []
                for row in channel:
                    trans_rows.append(row[1:].tolist() + [row[0]])
                trans_channels.append(trans_rows)
            elif direction == "up":
                tmp1 = channel[1:].tolist()
                tmp2 = channel[0].tolist()
                tmp1.append(tmp2)
                trans_channels.append(tmp1)
            elif direction == "down":
                tmp1 = [channel[-1].tolist()]
                tmp2 = channel[:-1].tolist()
                tmp1 = tmp1 + tmp2
                trans_channels.append(tmp1)
            
        trans_images.append(trans_channels)
    return np.asarray(trans_images, dtype=np.uint8)

In [ ]:
# Hmm. This is a bit repetitive... but oh well

In [47]:
orig_trans_bacon_data = []
refl_trans_bacon_data = []
for d in ["up", "down", "left", "right"]:
    orig_trans_bacon_data.append(translate(bacon_data, d))
    refl_trans_bacon_data.append(translate(refl_bacon_data, d))

In [48]:
orig_trans_kevin_data = []
refl_trans_kevin_data = []
for d in ["up", "down", "left", "right"]:
    orig_trans_kevin_data.append(translate(kevin_data, d))
    refl_trans_kevin_data.append(translate(refl_kevin_data, d))

In [49]:
orig_trans_not_data = []
refl_trans_not_data = []
for d in ["up", "down", "left", "right"]:
    orig_trans_not_data.append(translate(not_data, d))
    refl_trans_not_data.append(translate(refl_not_data, d))

In [50]:
all_bacon_data = np.asarray([bacon_data] + [refl_bacon_data] + orig_trans_bacon_data + refl_trans_bacon_data)
all_kevin_data = np.asarray([kevin_data] + [refl_kevin_data] + orig_trans_kevin_data + refl_trans_kevin_data)
all_not_data = np.asarray([not_data] + [refl_not_data] + orig_trans_not_data + refl_trans_not_data)

## Save generated data
So that we don't have to do this over an over.

In [51]:
all_bacon_data.tofile("generated_data/all_bacon_data.dat")
all_kevin_data.tofile("generated_data/all_kevin_data.dat")
all_not_data.tofile("generated_data/all_not_data.dat")

## Prepare data splits for training
Write the splits so that we have a static set of train/validation/test and don't have to do this all the time.

In [52]:
# Set seed for reproducibility across runs (e.g. for 64x64 vs. 128x128)
np.random.seed(99)

def shuffle_in_unison_inplace(a, b):
    p = np.random.permutation(len(a))
    return a[p], b[p]

def load_and_prepare_dataset():
    t_start = time.time()

    # Read in data
    data_bacon_file = open("generated_data/all_bacon_data.dat", "rb")
    data_kevin_file = open("generated_data/all_kevin_data.dat", "rb")
    data_not_file = open("generated_data/all_not_data.dat", "rb")
    bacon_data = np.fromfile(data_bacon_file, dtype=np.uint8)
    kevin_data = np.fromfile(data_kevin_file, dtype=np.uint8)
    not_data = np.fromfile(data_not_file, dtype=np.uint8)
    
    bacon_data = bacon_data.reshape((-1, 1, 128, 128))
    kevin_data = kevin_data.reshape((-1, 1, 128, 128))
    not_data = not_data.reshape((-1, 1, 128, 128))
    
    print("-- Files read")

    # Map classes to numbers
    y_bacon = np.empty(len(bacon_data), dtype=np.uint8)
    y_kevin = np.empty(len(kevin_data), dtype=np.uint8)
    y_not = np.empty(len(not_data), dtype=np.uint8)
    y_not.fill(0)
    y_bacon.fill(1)
    y_kevin.fill(2)

    # Coalesce the data
    X = np.concatenate((bacon_data, kevin_data))
    y = np.concatenate((y_bacon, y_kevin))
    print("-- Data coalesced")

    # Cast X data as floating point (single precision)
    X = X.astype(np.float32)

    # Randomize the order
    X, y = shuffle_in_unison_inplace(X, y)
    print("-- Data shuffled")

    # Break into 80:10:10 train:validation:test
    X_test = X[:(len(X) // 10)]
    X_val = X[(len(X) // 10):(2 * (len(X) // 10))]
    X_train = X[(2 * (len(X) // 10)):]

    y_test = y[:(len(y) // 10)]
    y_val = y[(len(y) // 10):(2 * (len(y) // 10))]
    y_train = y[(2 * (len(y) // 10)):]
    print("-- Data split")

    print("Time to load and prepare data: {}".format(time.time() - t_start))
    return X_train, y_train, X_val, y_val, X_test, y_test

In [53]:
X_train, y_train, X_val, y_val, X_test, y_test = load_and_prepare_dataset()

# Save sets to disk
X_train.tofile("generated_data/X_train.dat")
X_val.tofile("generated_data/X_val.dat")
X_test.tofile("generated_data/X_test.dat")

y_train.tofile("generated_data/y_train.dat")
y_val.tofile("generated_data/y_val.dat")
y_test.tofile("generated_data/y_test.dat")

-- Files read
-- Data coalesced
-- Data shuffled
-- Data split
Time to load and prepare data: 0.20145583152770996


In [54]:
len(y_train)

1600

In [5]:
y_bacon = np.empty(len(bacon_data), dtype=np.uint8)
y_kevin = np.empty(len(kevin_data), dtype=np.uint8)
y_not = np.empty(len(not_data), dtype=np.uint8)
y_not.fill(0)
y_bacon.fill(1)
y_kevin.fill(2)

In [6]:
len(bacon_data)

100

In [17]:
X = np.asarray([not_data[0], bacon_data[0], kevin_data[0]])
y = np.asarray([0, 1, 2])

In [19]:
XX, yy = shuffle_in_unison_inplace(X, y)

In [21]:
XX

array([[[[  0,  39,  64, ...,  33,  40,  27],
         [  0,  55,  86, ...,  37,  57, 118],
         [  0,  62,  91, ...,  73,  48, 133],
         ..., 
         [  0,  57,  87, ..., 154,  85, 157],
         [  0,  56,  86, ...,  64,  84,  71],
         [  0,  54,  86, ...,  60, 106, 174]],

        [[ 19,  78, 132, ...,  29,  38,  27],
         [  9,  92, 153, ...,  33,  55, 116],
         [  5,  96, 156, ...,  69,  45, 131],
         ..., 
         [  8,  91, 153, ..., 148,  74, 143],
         [  6,  89, 152, ...,  57,  71,  54],
         [  3,  87, 151, ...,  53,  93, 156]],

        [[ 14,  95, 177, ...,   0,   0,   0],
         [  3, 108, 195, ...,   0,  16,  78],
         [  0, 106, 194, ...,  22,   4,  92],
         ..., 
         [ 30, 118, 187, ..., 100,  28,  98],
         [ 32, 120, 187, ...,  11,  27,  11],
         [ 30, 120, 189, ...,   9,  51, 116]]],


       [[[ 22,  24,  28, ..., 206, 237, 203],
         [  9,  11,  20, ..., 211, 224, 214],
         [  3,   2,  13, ..

In [22]:
yy

array([0, 2, 1])

In [23]:
X

array([[[[  0,  39,  64, ...,  33,  40,  27],
         [  0,  55,  86, ...,  37,  57, 118],
         [  0,  62,  91, ...,  73,  48, 133],
         ..., 
         [  0,  57,  87, ..., 154,  85, 157],
         [  0,  56,  86, ...,  64,  84,  71],
         [  0,  54,  86, ...,  60, 106, 174]],

        [[ 19,  78, 132, ...,  29,  38,  27],
         [  9,  92, 153, ...,  33,  55, 116],
         [  5,  96, 156, ...,  69,  45, 131],
         ..., 
         [  8,  91, 153, ..., 148,  74, 143],
         [  6,  89, 152, ...,  57,  71,  54],
         [  3,  87, 151, ...,  53,  93, 156]],

        [[ 14,  95, 177, ...,   0,   0,   0],
         [  3, 108, 195, ...,   0,  16,  78],
         [  0, 106, 194, ...,  22,   4,  92],
         ..., 
         [ 30, 118, 187, ..., 100,  28,  98],
         [ 32, 120, 187, ...,  11,  27,  11],
         [ 30, 120, 189, ...,   9,  51, 116]]],


       [[[ 85, 123, 116, ..., 105, 109, 104],
         [ 98,  97, 129, ...,  88,  92, 112],
         [102,  89, 135, ..